In [1]:
import pandas as pd
import numpy as np
import datetime
import re

In [2]:
df = pd.read_csv('all_permit_v1.csv')

C:\Users\danie\anaconda_3\lib\site-packages\IPython\core\interactiveshell.py:3155: DtypeWarning: Columns (0,2,9,12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
df.head(3)

,structure_type,parcel,neighborhood,owner,issued,work description,permit,contractor,address,work_type,ward,id,cost,year,month,source,mn
0,NaN,0048s00284000000,NaN,pfohl donald e & lucy g (w),2012-01-30 00:00:00,interior fitout for yoga studio\n-pittsburgh b...,11-b-03000,nh construction llc,"3418 penn ave, pittsburgh, pa",NaN,6.0,NaN,200000.0,2012,january,xls,1
1,NaN,0006r00206000000,NaN,p7 investments lp,2012-01-04 00:00:00,new 3 story single family dwelling with 2 car ...,11-b-03019,heartland homes inc,"435 sweetbriar st, pittsburgh, pa",NaN,19.0,NaN,100000.0,2012,january,xls,1
2,NaN,0006r00206000000,NaN,dugan timothy,2012-01-04 00:00:00,new 3 story single family dwelling with 2 car ...,11-b-03021,heartland homes inc,"437 sweetbriar st 435, pittsburgh, pa",NaN,19.0,NaN,100000.0,2012,january,xls,1


In [4]:
df.dtypes

structure_type       object
parcel               object
neighborhood         object
owner                object
issued               object
work description     object
permit               object
contractor           object
address              object
work_type            object
ward                 object
id                  float64
cost                 object
year                  int64
month                object
source               object
mn                    int64
dtype: object

In [5]:
# Missing data %
df.isna().sum()/len(df)*100

structure_type      78.356972
parcel               0.066002
neighborhood        80.891729
owner                0.022469
issued              24.110378
work description     7.649207
permit               0.000000
contractor          17.529841
address              0.000000
work_type           82.217385
ward                 1.216121
id                  27.510181
cost                 0.188176
year                 0.000000
month                0.000000
source               1.475916
mn                   0.000000
dtype: float64

In [73]:
# Drop missing cost, send these to a file for later inspection
no_cost = df[df.cost.isna()]

In [74]:
no_cost.to_csv('Missing fields/cost_na.csv',index=False)

In [6]:
df = df[~df.cost.isna()]

In [77]:
# Send "no primary address listed" out to file
no_address = df[df.address.str.contains('no primary')]

In [78]:
no_address.to_csv('Missing fields/address_na.csv',index=False)

In [7]:
df = df[~df.address.str.contains('no primary')]

In [30]:
# Missing data %
df.isna().sum()/len(df)*100

structure_type      78.431759
parcel               0.066165
neighborhood        80.971352
owner                0.022524
issued              24.165552
work description     7.662420
permit               0.000000
contractor          17.567396
address              0.000000
work_type           82.252411
ward                 1.213486
id                  27.577955
cost                 0.000000
year                 0.000000
month                0.000000
source               1.479552
mn                   0.000000
clean_ward           1.213486
dtype: float64

In [8]:
# Add pittsburgh to addresses with only streets
city = ', pittsburgh, pa'
f = lambda x: x+city if ',' not in x else x
df['address'] = df['address'].apply(lambda x:f(x))

In [9]:
df[~df.address.str.contains('pittsburgh')&~df.address.str.contains(' pa')]

,structure_type,parcel,neighborhood,owner,issued,work description,permit,contractor,address,work_type,ward,id,cost,year,month,source,mn
3716,NaN,0003k00161000000,NaN,windom hill place lp,2013-06-03t00:00:00,10ft 3inches high retaining wall on easterly s...,13-b-01323,"sota construction services, inc.","22 windom hill pl, unit",NaN,17.0,7.0,4800,2013,june,API,6
53621,NaN,0023r00157000000,NaN,allegheny city,2019-04-04t00:00:00,interior renovation of 960 sq ft on the ground...,19-b-00237,mascaro construction company lp,"6 allegheny square east, #101",NaN,22,151.0,149298,2019,april,API,4


In [10]:
df.loc[53621, 'address'] = '6 allegheny square east, #101, pittsburgh, pa'

In [11]:
df.loc[3716, 'address'] = '22 windom hill pl, pittsburgh, pa'

In [12]:
misspelled = df[df.address.str.contains('pittsbugrh')].index.values

In [13]:
for i in misspelled:
    df.loc[i,'address'] = df.loc[i,'address'].replace('pittsbugrh','pittsburgh')

In [14]:
df.duplicated().sum()

0

In [15]:
df[df.iloc[:,10].str.contains(r'[A-Za-z]')==True]

,structure_type,parcel,neighborhood,owner,issued,work description,permit,contractor,address,work_type,ward,id,cost,year,month,source,mn
9656,NaN,parcel,NaN,owner,NaN,work description,permit number,contractor,"address, pittsburgh, pa",NaN,ward,168.0,cost,2015,august,API,5


In [16]:
df.drop(index=9656, inplace = True)

In [17]:
# Clean wards 
df.ward.unique()

array(['6.0', '19.0', '15.0', '4.0', '1.0', '2.0', '29.0', '18.0', '31.0',
       '24.0', '14.0', '12.0', '20.0', '26.0', '10.0', '17.0', '13.0',
       '3.0', '22.0', '5.0', '21.0', '16.0', '7.0', '32.0', '11.0', '9.0',
       '27.0', '30.0', '23.0', '8.0', '25.0', '28.0', nan, '7', '14', '2',
       '5', '30', '1', '27', '13', '22', '12', '6', '11', '9', '19', '16',
       '4', '8', '15', '17', '10', '23', '20', '32', '31', '21', '25',
       '28', '24', '26', '29', '18', '3', '5/5', '2820', '19;20', '19;17',
       '26;25', '15;17', '10;6', '11;12', '21;22', '6;2', '9;14',
       '10;11;12', '21;27', '5;4', '24;23', '3;2', '18;29;32', '29;32',
       '19;32', '10;9', '18;30', '14;15', '20;28', '30;29', '6;9', '9;6',
       '2;10', '26;27'], dtype=object)

In [18]:
# Just keep the first one noted
def fix_ward(x):
    if pd.isna(x):
        return np.nan    
    elif ';' in x:
        return x.split(';')[0]
    elif '/' in x:
        return x.split('/')[0]
    elif '.' in x:
        return x.split('.')[0]
    elif '2820' in x:
        return '28'
    else:
        return x     

In [19]:
df['clean_ward'] = df['ward'].apply(lambda x:fix_ward(x))

In [20]:
df['cost'] = df['cost'].astype('float')

In [21]:
df.describe()

,id,cost,year,mn
count,51445.000000,7.103500e+04,71035.000000,71035.000000
mean,434.795238,1.196324e+05,2017.052946,6.441402
std,303.099722,1.538825e+06,2.131301,3.354115
min,1.000000,0.000000e+00,2012.000000,1.000000
25%,170.000000,1.500000e+03,2016.000000,4.000000
50%,387.000000,6.557000e+03,2017.000000,6.000000
75%,672.000000,2.500000e+04,2019.000000,10.000000
max,1328.000000,1.900000e+08,2021.000000,12.000000


In [22]:
# Begin address cleaning, look for symbols
df[df['address'].str.findall(r'[^#a-zA-Z0-9, /]').str[0].notna()]

,structure_type,parcel,neighborhood,owner,issued,work description,permit,contractor,address,work_type,ward,id,cost,year,month,source,mn,clean_ward
5621,NaN,0001c01662012500,NaN,ryan keefe,2014-01-24t00:00:00,interior condo renovations-12e,14-b-00143,randy stebler general contracting,"320 fort duquesne blvd 12-e, pittsburgh, pa",NaN,2.0,168.0,60000.0,2014,january,API,1,2
5721,NaN,0001c01662f01900,NaN,betancourt carlos,2014-02-28t00:00:00,condo renovations,14-b-00106,fjw investment inc cabinet world,"320 fort duquesne blvd 19-f, pittsburgh, pa",NaN,2,77.0,22000.0,2014,february,API,2,2
6991,NaN,0001c01662h01200,NaN,anne and mike osborn,2014-06-06t00:00:00,interior renovations to condo unit 12 h on 12t...,14-b-01256,delaney's decorating,"320 fort duquesne blvd 12-h, pittsburgh, pa",NaN,2,114.0,120000.0,2014,june,API,6,2
7066,NaN,0001c01662009800,NaN,barone alvin j jr,2014-06-13t00:00:00,interior renovations to kitchen in condo unit ...,14-b-01776,renovision,"320 fort duquesne blvd 9-h, pittsburgh, pa",NaN,2,189.0,20000.0,2014,june,API,6,2
8164,NaN,0001c01662111300,NaN,coogan david & leigh argentieri,2014-10-21t00:00:00,interior renovations to condo unit 11m in 27 s...,14-b-03439,renovision,"320 fort duquesne blvd 11-m, pittsburgh, pa",NaN,2,243.0,14000.0,2014,october,API,10,2
8829,NaN,0001c01662021300,NaN,"lilan laishley, william harmon",2015-01-08t00:00:00,interior renovations to kitchen and bath in co...,15-b-00040,renovision,"320 fort duquesne blvd 21-c, pittsburgh, pa",NaN,2.0,85.0,13000.0,2015,january,API,1,2
8920,NaN,0001c01662020900,NaN,miller harold d & sharon e (w),2015-01-08t00:00:00,interior renovations to bath in condo 20j in 2...,15-b-00041,renovision,"320 fort duquesne blvd 20-i, pittsburgh, pa",NaN,2.0,177.0,10000.0,2015,january,API,1,2
9321,NaN,0001c01662009300,NaN,keefe timothy j,2015-03-13t00:00:00,interior renovations to kitchen and bath in un...,15-b-00621,randy stebler general contracting,"320 fort duquesne blvd 9-c, pittsburgh, pa",NaN,2,205.0,20000.0,2015,march,API,3,2
12576,NaN,0001c01662011200,NaN,burstein robert h,NaN,interior demolition to unit 24c in 25 story st...,15-b-02986,robert hill,"320 fort duquesne blvd, #24-c, pittsburgh, pa ...",NaN,2,656.0,3000.0,2015,september,API,9,2
12577,NaN,0001c01662024300,NaN,burstein robert h,NaN,interior demolition to unit 24c in 25 story st...,15-b-02986,robert hill,"320 fort duquesne blvd, #24-c, pittsburgh, pa ...",NaN,2,657.0,3000.0,2015,september,API,9,2


In [23]:
# Prefer just single # to refer to anything regarding "unit" as this is common and known by Geocoders
df['address'] = df['address'].str.replace('#unit','#')

In [24]:
df['address'] = df['address'].str.replace('##','#')

In [25]:
# Find digit-letter units and kill the dash, i.e., make 12-e into 12e
pat = r'([0-9][-][a-zA-Z])'
df[df['address'].str.contains(pat)]

C:\Users\danie\anaconda_3\lib\site-packages\pandas\core\strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


,structure_type,parcel,neighborhood,owner,issued,work description,permit,contractor,address,work_type,ward,id,cost,year,month,source,mn,clean_ward
5621,NaN,0001c01662012500,NaN,ryan keefe,2014-01-24t00:00:00,interior condo renovations-12e,14-b-00143,randy stebler general contracting,"320 fort duquesne blvd 12-e, pittsburgh, pa",NaN,2.0,168.0,60000.0,2014,january,API,1,2
5721,NaN,0001c01662f01900,NaN,betancourt carlos,2014-02-28t00:00:00,condo renovations,14-b-00106,fjw investment inc cabinet world,"320 fort duquesne blvd 19-f, pittsburgh, pa",NaN,2,77.0,22000.0,2014,february,API,2,2
6991,NaN,0001c01662h01200,NaN,anne and mike osborn,2014-06-06t00:00:00,interior renovations to condo unit 12 h on 12t...,14-b-01256,delaney's decorating,"320 fort duquesne blvd 12-h, pittsburgh, pa",NaN,2,114.0,120000.0,2014,june,API,6,2
7066,NaN,0001c01662009800,NaN,barone alvin j jr,2014-06-13t00:00:00,interior renovations to kitchen in condo unit ...,14-b-01776,renovision,"320 fort duquesne blvd 9-h, pittsburgh, pa",NaN,2,189.0,20000.0,2014,june,API,6,2
8164,NaN,0001c01662111300,NaN,coogan david & leigh argentieri,2014-10-21t00:00:00,interior renovations to condo unit 11m in 27 s...,14-b-03439,renovision,"320 fort duquesne blvd 11-m, pittsburgh, pa",NaN,2,243.0,14000.0,2014,october,API,10,2
8829,NaN,0001c01662021300,NaN,"lilan laishley, william harmon",2015-01-08t00:00:00,interior renovations to kitchen and bath in co...,15-b-00040,renovision,"320 fort duquesne blvd 21-c, pittsburgh, pa",NaN,2.0,85.0,13000.0,2015,january,API,1,2
8920,NaN,0001c01662020900,NaN,miller harold d & sharon e (w),2015-01-08t00:00:00,interior renovations to bath in condo 20j in 2...,15-b-00041,renovision,"320 fort duquesne blvd 20-i, pittsburgh, pa",NaN,2.0,177.0,10000.0,2015,january,API,1,2
9321,NaN,0001c01662009300,NaN,keefe timothy j,2015-03-13t00:00:00,interior renovations to kitchen and bath in un...,15-b-00621,randy stebler general contracting,"320 fort duquesne blvd 9-c, pittsburgh, pa",NaN,2,205.0,20000.0,2015,march,API,3,2
12576,NaN,0001c01662011200,NaN,burstein robert h,NaN,interior demolition to unit 24c in 25 story st...,15-b-02986,robert hill,"320 fort duquesne blvd, #24-c, pittsburgh, pa ...",NaN,2,656.0,3000.0,2015,september,API,9,2
12577,NaN,0001c01662024300,NaN,burstein robert h,NaN,interior demolition to unit 24c in 25 story st...,15-b-02986,robert hill,"320 fort duquesne blvd, #24-c, pittsburgh, pa ...",NaN,2,657.0,3000.0,2015,september,API,9,2


In [26]:
df[df['address'].str.contains(pat)].index.values

C:\Users\danie\anaconda_3\lib\site-packages\pandas\core\strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


array([ 5621,  5721,  6991,  7066,  8164,  8829,  8920,  9321, 12576,
       12577, 13997, 14997, 16210, 17890, 27750, 44961, 50234, 53579,
       60115, 61051, 61184], dtype=int64)

In [28]:
for i in df[df['address'].str.contains(pat)].index.values:
    df.loc[i,'address'] = df.loc[i,'address'].replace('-','')

In [29]:
# Find addresses with uncommon symbols
df[df['address'].str.findall(r'[^#a-zA-Z0-9, /]').str[0].notna()]

,structure_type,parcel,neighborhood,owner,issued,work description,permit,contractor,address,work_type,ward,id,cost,year,month,source,mn,clean_ward
16013,NaN,0016r00156000000,NaN,board of public education of the schl dist of ...,2/8/2016,to replace permit #14-f-00188. see #14-f-00188...,16-f-00034,NaN,"810 rockland ave, pittsburgh< pa, pa 15216",NaN,19.0,NaN,0.0,2016,february,xslx,2,19
20995,NaN,0016m00100000000,NaN,school district of pittsburgh,6/7/2016,fire alarm alteration,16-f-00181,right electric,"590 -595 crane ave, pittsburgh, pa 15216",NaN,19.0,NaN,9000.0,2016,june,xslx,6,19
21972,NaN,0001h00256000000,NaN,pnc bank national association,7/6/2016,work related to 16-b-00517 relocation of offic...,16-f-00209,lighthouse electric,"300 -314 5th ave, pittsburgh, pa 15222",NaN,2.0,NaN,30000.0,2016,july,xslx,7,2
22034,NaN,0016m00100000000,NaN,school district of pittsburgh,7/7/2016,install outlets,16-e-01772,wheels mechanical,"590 -595 crane ave, pittsburgh, pa 15216",NaN,19.0,NaN,700.0,2016,july,xslx,7,19
22062,NaN,0056k00248000000,NaN,hazelwood second ave lp,7/8/2016,reconnect,16-e-01805,NaN,"5011 -5013 2nd ave, pittsburgh, pa 15207",NaN,15.0,NaN,500.0,2016,july,xslx,7,15
22454,NaN,0126d00236000000,NaN,darnall lamoyne william jr,7/18/2016,total rewire,16-e-01926,NaN,"205 -207 n homewood ave, pittsburgh, pa 15208",NaN,14.0,NaN,8000.0,2016,july,xslx,7,14
23247,NaN,134757,NaN,penn liberty holding company,9/13/2016,generators for event,16-e-02566,NaN,"penn & 7th, pittsburgh, pa",NaN,2.0,NaN,450.0,2016,september,xslx,9,2
23918,NaN,0023l00080000000,NaN,coa north side llc,NaN,install hood supression system,16-s-00267,fire control serv. inc.,"40 -44 w north ave, pittsburgh, pa 15212",NaN,22,17.0,0.0,2016,october,API,10,22
45809,NaN,0027h00074p00300,NaN,robert and loretta barone,2018-06-27t00:00:00,fixtures and outlets related to 18-b-01938,18-e-01733,schreckengost electric,"146 n bellefield ave, #ph-3, pittsburgh, pa 15213",NaN,7,884.0,16000.0,2018,june,API,6,7
66923,commercial - all other uses,0025-f-00177-00g3-00,strip district,berger lee j & meghan f (w),2020-07-21t00:00:00,add new sprinkler heads to existing system.,ssp-2020-10123,metropolitan fire protection,"2901 g-3 smallman st, pittsburgh, pa 15201",NaN,6,578.0,2000.0,2020,july,API,7,6


In [31]:
# Narrow change for weird address
df.loc[16013,'address'] = df.loc[15865,'address'].replace('< pa','')

In [32]:
# Narrow change for weird address
df.loc[45809,'address'] = df.loc[45661,'address'].replace('#ph-3','# ph3')

In [33]:
# Narrow change for weird address
df.loc[66923,'address'] = df.loc[66775,'address'].replace('g-3','g3')

In [34]:
# Keep first address when address refers to several units
multiaddress = df[df['address'].str.findall(r'[^#a-zA-Z0-9, /]').str[0].notna()].index.values

In [35]:
def take_first(x):
    if " -" in x:
        y = ' '+x.split()[1]
        first = x.replace(y, '')
        return first
    else:
        return x

In [36]:
# Take addresses like "100 -105 n avenue" and just make them "100 n avenue" for simplicity
for addr in multiaddress:
    df.loc[addr, 'address'] = take_first(df.loc[addr, 'address'])

In [37]:
df[df['address'].str.findall(r'[^#a-zA-Z0-9, /]').str[0].notna()]

,structure_type,parcel,neighborhood,owner,issued,work description,permit,contractor,address,work_type,ward,id,cost,year,month,source,mn,clean_ward
23247,NaN,134757,NaN,penn liberty holding company,9/13/2016,generators for event,16-e-02566,NaN,"penn & 7th, pittsburgh, pa",NaN,2.0,NaN,450.0,2016,september,xslx,9,2


In [38]:
# Just drop this outlier - very small, not clear address
df.drop(index=23247, inplace=True)

In [39]:
# Look for two+ white spaces to address gaps
df[df.loc[:,'address'].str.contains(r'([ ]){2,}')]

C:\Users\danie\anaconda_3\lib\site-packages\pandas\core\strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


,structure_type,parcel,neighborhood,owner,issued,work description,permit,contractor,address,work_type,ward,id,cost,year,month,source,mn,clean_ward
52606,NaN,0123n00322000000,NaN,harding john daniel jr & emilykaufmann (w),2019-03-06 00:00:00,construction of soil nail retaining wall and i...,18-l-00015,geostabilization international llc,"6512 jackson st, , pittsburgh, pa 15206",NaN,11.0,NaN,300000.0,2019,march,NaN,3,11
56261,NaN,0002m00065000000,NaN,mercy hospital of pittsburgh,2019-05-21t00:00:00,install fixtures on existing branch circuits,19-e-00656,hanlon electric company,"1401 blvd of the allies , , pittsburgh...",NaN,1.0,614.0,25900.0,2019,may,API,5,1


In [40]:
# Only 2! just change in line
df.loc[52606,'address'] = '6512 jackson st, pittsburgh, pa 15206'

In [41]:
df.loc[56261,'address'] = '1401 blvd of the allies, pittsburgh, pa 15219'

In [43]:
# clear hanging white space
df.loc[:,'address'] = df.loc[:,'address'].str.replace(r' $', '')

In [44]:
df.loc[:,'address'].str.contains(r' $').sum()

0

In [45]:
# likely abbreviations flanked by commas or whitespace
full = {' rd,':' road,', ' ave,':' avenue,',' st,':' street,', ' blvd,':' boulevard,',
 ' ct,':' court,', ' pl,':' place,', ' dr,':' drive,', ' ln,':' lane,', ' ctr,': ' center,',
       ' rd ':' road,', ' ave ':' avenue ',' st ':' street ', ' blvd ':' boulevard,',
 ' ct ':' court,', ' pl ':' place,', ' dr ':' drive,', ' ln ':' lane,', ' ctr ': ' center,'}

In [46]:
def complete_street(x):
    for i in full.keys():
        if i in x:
            x = x.replace(i, full[i])
    return x

In [47]:
# likely abbreviations flanked by commas or whitespace
direc = {' n ':' north ', ' e ':' east ',' s ':' south ', ' w ':' west '}

In [48]:
def complete_direc(x):
    for i in direc.keys():
        if i in x:
            x = x.replace(i, direc[i])
    return x

In [49]:
# Apply the func to cleanse abbreviations
df.loc[:,'address'] = df.loc[:,'address'].apply(lambda x: complete_street(x))

In [50]:
# Apply the func to cleanse abbreviations
df.loc[:,'address'] = df.loc[:,'address'].apply(lambda x: complete_direc(x))

In [51]:
nth = {' 1st':' first', ' 2nd':' second',' 3rd':' third', ' 4th':' fourth', ' 5th':' fifth',
      ' 6th':' sixth', ' 7th':' seventh', ' 8th':' eighth', ' 9th':' ninth'}

In [52]:
def complete_nth(x):
    for i in nth.keys():
        if i in x:
            x = x.replace(i, nth[i])
    return x

In [53]:
df.loc[:,'address'] = df.loc[:,'address'].apply(lambda x: complete_nth(x))

In [54]:
df.loc[:,'address']

0                     3418 penn avenue, pittsburgh, pa
1                435 sweetbriar street, pittsburgh, pa
2            437 sweetbriar street 435, pittsburgh, pa
3           4656 browns hill road,4634, pittsburgh, pa
4                    4400 fifth avenue, pittsburgh, pa
                             ...                      
71205          323 denniston street, pittsburgh, 15206
71206            211 26th street, pittsburgh, pa 15222
71207              5900 penn avenue, pittsburgh, 15206
71208    211 north whitfield street, pittsburgh, 15206
71209          323 denniston street, pittsburgh, 15206
Name: address, Length: 71034, dtype: object

In [55]:
# Address column warnings
df.iloc[:,[0,2,9]]

,structure_type,neighborhood,work_type
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN
...,...,...,...
71205,residential,shadyside,addition / alteration
71206,commercial,strip district,addition / alteration
71207,commercial,east liberty,addition / alteration
71208,commercial,east liberty,minor alteration


In [56]:
df.iloc[:,0].replace(np.nan,'not_listed')

0         not_listed
1         not_listed
2         not_listed
3         not_listed
4         not_listed
            ...     
71205    residential
71206     commercial
71207     commercial
71208     commercial
71209    residential
Name: structure_type, Length: 71034, dtype: object

In [57]:
for i in [0,2,9]:
    df.iloc[:,i]= df.iloc[:,i].replace(np.nan,'not_listed')

In [58]:
pat = r'^[a-zA-Z 0,]'

In [60]:
# Ah, addresses with 0... 
df[df.loc[:,'address'].str.contains(pat)]

,structure_type,parcel,neighborhood,owner,issued,work description,permit,contractor,address,work_type,ward,id,cost,year,month,source,mn,clean_ward
662,not_listed,0089a00116000000,not_listed,equitable gas company,2012-05-17 00:00:00,installation of one antenna at a height of 345...,11-b-02408,jg contracting company inc,"0 manitoba street, pittsburgh, pa",not_listed,15.0,10.0,3105.0,2012,may,API,5,15
1930,not_listed,0048f00390000000,not_listed,city of pittsburgh,2012-10-02 00:00:00,replace decking on overlook located on concret...,12-b-02397,kucich construction,"0 herrs island, pittsburgh, pa",not_listed,24.0,NaN,55000.0,2012,october,xls,10,24
2318,not_listed,0125k00005000000,not_listed,port authority of allegheny county,2012-12-13 00:00:00,NaN,12-b-02649,massaro restoration services llc,"fifth avenue, pittsburgh, pa",not_listed,12.0,NaN,250000.0,2012,december,xls,12,12
2487,not_listed,0015c00082000000,not_listed,port authority of allegheny county,1/16/2013,install structural support for 11 exhaust fans,12-b-03046,the chappy corporation,"0 east warrington avenue, pittsburgh, pa",not_listed,19.0,NaN,0.0,2013,january,xslx,1,19
3673,not_listed,0023r00202000002,not_listed,bell telephone company of pennsylvania,2013-05-29t00:00:00,installation of a telecommunication operations...,13-b-01202,sargent electric co,"0 montgomery place, pittsburgh, pa",not_listed,22.0,265.0,300000.0,2013,may,API,5,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51309,not_listed,0051e00204000000,not_listed,ross craig s,2019-01-24t00:00:00,"remove wooden doors, knock down brick walls an...",18-d-00235,smuts brothers debris removal,"0 lima way, pittsburgh, pa 15224",not_listed,8.0,707.0,2500.0,2019,january,API,1,8
52305,not_listed,0077p00083000000,not_listed,streb raymond a,2019-02-26t00:00:00,install 100a service,19-e-00546,king electric,"0 sherlock street, pittsburgh, pa 15214",not_listed,26.0,806.0,2200.0,2019,february,API,2,26
54225,not_listed,0002h00289000001,not_listed,r r properties llc,2019-04-22t00:00:00,demolition of a three story church,19-d-00019,noralco corporation,"0 vine street, pittsburgh, pa 15219",not_listed,3,753.0,130000.0,2019,april,API,4,3
54495,not_listed,0026j00181000000,not_listed,3357 ridgeway llc,2019-04-29t00:00:00,occ only,19-occ-00302,NaN,"0 bigelow boulevard, pittsburgh, pa",not_listed,6,1025.0,0.0,2019,april,API,4,6


In [59]:
df[df.loc[:,'address'].str.contains(pat)].index.values

array([  662,  1930,  2318,  2487,  3673,  7340,  8090,  9975, 11070,
       11737, 11895, 12495, 12542, 12788, 13160, 13293, 13432, 13502,
       13524, 13525, 13843, 14018, 14080, 14192, 14210, 14817, 17339,
       18039, 18387, 18403, 18410, 18427, 18671, 20144, 20758, 21026,
       21030, 21127, 21214, 21288, 21455, 21548, 21628, 21700, 21712,
       21728, 21733, 22235, 22738, 23057, 25101, 26608, 28336, 28517,
       29407, 30906, 30907, 31587, 32487, 32488, 32542, 33695, 33757,
       34120, 34808, 36053, 37438, 39673, 39920, 47525, 51309, 52305,
       54225, 54495, 57714], dtype=int64)

In [16]:
df[df.loc[:,'address'].str.contains(pat)].to_csv('Missing fields/addr_number_na.csv',index=False)

In [61]:
df = df[~df.loc[:,'address'].str.contains(pat)]

In [62]:
len(df)

70959

## Split addresses to see about consolidating where possible

In [64]:
df.head(2)

,structure_type,parcel,neighborhood,owner,issued,work description,permit,contractor,address,work_type,ward,id,cost,year,month,source,mn,clean_ward
0,not_listed,0048s00284000000,not_listed,pfohl donald e & lucy g (w),2012-01-30 00:00:00,interior fitout for yoga studio\n-pittsburgh b...,11-b-03000,nh construction llc,"3418 penn avenue, pittsburgh, pa",not_listed,6.0,NaN,200000.0,2012,january,xls,1,6
1,not_listed,0006r00206000000,not_listed,p7 investments lp,2012-01-04 00:00:00,new 3 story single family dwelling with 2 car ...,11-b-03019,heartland homes inc,"435 sweetbriar street, pittsburgh, pa",not_listed,19.0,NaN,100000.0,2012,january,xls,1,19


In [65]:
df['split_addr'] = df.loc[:,'address'].str.split(',')

In [66]:
x = 0
for i in df['address'].str.split(','):
    if len(i) > x:
        x = len(i)

In [67]:
x

4

In [68]:
df['addr1'] = df.loc[:,'split_addr'].apply(lambda x: x[0] if len(x)>0 else '')

In [69]:
df['addr2'] = df.loc[:,'split_addr'].apply(lambda x: x[1] if len(x)>1 else '')
df['addr3'] = df.loc[:,'split_addr'].apply(lambda x: x[2] if len(x)>2 else '')
df['addr4'] = df.loc[:,'split_addr'].apply(lambda x: x[3] if len(x)>3 else '')

In [70]:
sorted(df.addr2.unique())

[' ',
 ' #1',
 ' #1004',
 ' #1006',
 ' #101',
 ' #1015',
 ' #102',
 ' #103',
 ' #104',
 ' #105',
 ' #1250',
 ' #13',
 ' #1404',
 ' #1415',
 ' #142',
 ' #14e',
 ' #15',
 ' #15212',
 ' #15c',
 ' #165',
 ' #17',
 ' #175',
 ' #17o',
 ' #18',
 ' #1801',
 ' #1902',
 ' #19a',
 ' #2',
 ' #20',
 ' #2001',
 ' #201',
 ' #202',
 ' #203',
 ' #204',
 ' #206',
 ' #21',
 ' #210',
 ' #21f',
 ' #21m',
 ' #22',
 ' #23d',
 ' #24c',
 ' #24h',
 ' #25',
 ' #28',
 ' #2b',
 ' #2p',
 ' #3',
 ' #301',
 ' #302',
 ' #303',
 ' #304',
 ' #306',
 ' #308',
 ' #310',
 ' #311',
 ' #33',
 ' #3a',
 ' #3b',
 ' #3p',
 ' #4',
 ' #401',
 ' #402',
 ' #403',
 ' #406',
 ' #408',
 ' #411',
 ' #412',
 ' #414',
 ' #450',
 ' #4a',
 ' #4jk',
 ' #5',
 ' #503',
 ' #505',
 ' #509',
 ' #512',
 ' #519',
 ' #55',
 ' #57',
 ' #6',
 ' #601',
 ' #602',
 ' #604',
 ' #606',
 ' #615',
 ' #616',
 ' #6g',
 ' #6k',
 ' #7',
 ' #701',
 ' #705',
 ' #707',
 ' #711',
 ' #716',
 ' #717',
 ' #7e',
 ' #8',
 ' #806',
 ' #81',
 ' #840c',
 ' #850c',
 ' #8k',


In [89]:
# isolate addr2 components that need fixing
df[df.loc[:,'addr2'].str.contains(r'^[a-z]')]

,structure_type,parcel,neighborhood,owner,issued,work description,permit,contractor,address,work_type,...,year,month,source,mn,clean_ward,split_addr,addr1,addr2,addr3,addr4
2592,not_listed,0088m00100000000,not_listed,urban redevelopment auth of pittsburgh,2013-02-06 00:00:00,"new, 2 story four (4) unit condo building (uni...",12-b-01750,kacin a richard inc,"1614 overton lane, pittsburgh, pa",not_listed,...,2013,february,API,2,14,"[1614 overton lane, a, pittsburgh, pa]",1614 overton lane,a,pittsburgh,pa
8947,not_listed,0049c00158000000,not_listed,waterford land partners lp,2015-02-03t00:00:00,new construction of 2 1/2 story single family ...,14-b-02104,infinity custom homes,"323 winesap drive, pittsburgh, pa",not_listed,...,2015,february,API,2,9,"[323 winesap drive, u, pittsburgh, pa]",323 winesap drive,u,pittsburgh,pa
8948,not_listed,0049c00158000000,not_listed,waterford land partners lp,2015-02-03t00:00:00,new construction of 2 1/2 story single family ...,14-b-02105,infinity custom homes,"319 winesap drive, pittsburgh, pa",not_listed,...,2015,february,API,2,9,"[319 winesap drive, t, pittsburgh, pa]",319 winesap drive,t,pittsburgh,pa
8949,not_listed,0049c00158000000,not_listed,waterford land partners lp,2015-02-03t00:00:00,new construction of 2 1/2 story single family ...,14-b-02109,infinity custom homes,"309 winesap drive, pittsburgh, pa",not_listed,...,2015,february,API,2,9,"[309 winesap drive, q, pittsburgh, pa]",309 winesap drive,q,pittsburgh,pa
8950,not_listed,0049c00158000000,not_listed,waterford land partners lp,2015-02-03t00:00:00,new construction of 2 1/2 story single family ...,14-b-02086,infinity custom homes,"343 winesap drive, pittsburgh, pa",not_listed,...,2015,february,API,2,9,"[343 winesap drive, y, pittsburgh, pa]",343 winesap drive,y,pittsburgh,pa
8951,not_listed,0049c00158000000,not_listed,waterford land partners lp,2015-02-03t00:00:00,new construction of 2 1/2 story single family ...,14-b-02103,infinity custom homes,"335 winesap drive, pittsburgh, pa",not_listed,...,2015,february,API,2,9,"[335 winesap drive, w, pittsburgh, pa]",335 winesap drive,w,pittsburgh,pa
8953,not_listed,0049c00158000000,not_listed,waterford land partners lp,2015-02-03t00:00:00,new construction of 2 1/2 story single family ...,14-b-02107,infinity custom homes,"315 winesap drive, pittsburgh, pa",not_listed,...,2015,february,API,2,9,"[315 winesap drive, s, pittsburgh, pa]",315 winesap drive,s,pittsburgh,pa
8954,not_listed,0049c00158000000,not_listed,waterford land partners lp,2015-02-03t00:00:00,new construction of 2 1/2 story single family ...,14-b-02102,infinity custom homes,"339 winesap drive, pittsburgh, pa",not_listed,...,2015,february,API,2,9,"[339 winesap drive, x, pittsburgh, pa]",339 winesap drive,x,pittsburgh,pa
8955,not_listed,0049c00158000000,not_listed,waterford land partners lp,2015-02-03t00:00:00,new construction of 2 1/2 story single family ...,14-b-02108,infinity custom homes,"311 winesap drive, pittsburgh, pa",not_listed,...,2015,february,API,2,9,"[311 winesap drive, r, pittsburgh, pa]",311 winesap drive,r,pittsburgh,pa
8956,not_listed,0049c00158000000,not_listed,waterford land partners lp,2015-02-03t00:00:00,new construction of 2 1/2 story single family ...,14-b-02111,infinity custom homes,"301 winesap drive, pittsburgh, pa",not_listed,...,2015,february,API,2,9,"[301 winesap drive, o, pittsburgh, pa]",301 winesap drive,o,pittsburgh,pa


In [63]:
df.loc[:,'address'] = df.loc[:,'address'].str.replace('boulevard,of the allies,', 'boulevard of the allies,')

In [88]:
# Get rid of these 'a' and 'cc' type additions, they mean nothing to addresses
df.loc[:,'address'] = df.loc[:,'address'].str.replace(r',[a-z]{1},', ',')

In [73]:
df.loc[:,'address'] = df.loc[:,'address'].str.replace(r',[a-z]{2},', ',')

In [74]:
# Narrow fixes for things I probably messed up (drive, blvd...)
fixes = ['boulevard,of the allies avenue,',
'center,center,',
'boulevard,of the allies boulevard,',
'drive,drive,']

In [75]:
df.loc[:,'address'] = df.loc[:,'address'].str.replace(fixes[0], 'boulevard of the allies,')
df.loc[:,'address'] = df.loc[:,'address'].str.replace(fixes[1], 'center,')
df.loc[:,'address'] = df.loc[:,'address'].str.replace(fixes[2], 'boulevard of the allies,')
df.loc[:,'address'] = df.loc[:,'address'].str.replace(fixes[3], 'drive,')

In [76]:
sorted(df.addr2.unique())

[' ',
 ' #1',
 ' #1004',
 ' #1006',
 ' #101',
 ' #1015',
 ' #102',
 ' #103',
 ' #104',
 ' #105',
 ' #1250',
 ' #13',
 ' #1404',
 ' #1415',
 ' #142',
 ' #14e',
 ' #15',
 ' #15212',
 ' #15c',
 ' #165',
 ' #17',
 ' #175',
 ' #17o',
 ' #18',
 ' #1801',
 ' #1902',
 ' #19a',
 ' #2',
 ' #20',
 ' #2001',
 ' #201',
 ' #202',
 ' #203',
 ' #204',
 ' #206',
 ' #21',
 ' #210',
 ' #21f',
 ' #21m',
 ' #22',
 ' #23d',
 ' #24c',
 ' #24h',
 ' #25',
 ' #28',
 ' #2b',
 ' #2p',
 ' #3',
 ' #301',
 ' #302',
 ' #303',
 ' #304',
 ' #306',
 ' #308',
 ' #310',
 ' #311',
 ' #33',
 ' #3a',
 ' #3b',
 ' #3p',
 ' #4',
 ' #401',
 ' #402',
 ' #403',
 ' #406',
 ' #408',
 ' #411',
 ' #412',
 ' #414',
 ' #450',
 ' #4a',
 ' #4jk',
 ' #5',
 ' #503',
 ' #505',
 ' #509',
 ' #512',
 ' #519',
 ' #55',
 ' #57',
 ' #6',
 ' #601',
 ' #602',
 ' #604',
 ' #606',
 ' #615',
 ' #616',
 ' #6g',
 ' #6k',
 ' #7',
 ' #701',
 ' #705',
 ' #707',
 ' #711',
 ' #716',
 ' #717',
 ' #7e',
 ' #8',
 ' #806',
 ' #81',
 ' #840c',
 ' #850c',
 ' #8k',


In [77]:
# has 'pittsburgh,' but not PA 
noPA = df[~df.loc[:,'address'].str.contains(' pa') & df.loc[:,'address'].str.contains('pittsburgh,')].index.values

In [78]:
noPA

array([54603, 54604, 54605, ..., 71207, 71208, 71209], dtype=int64)

In [79]:
df.loc[noPA[0],'address'].replace('pittsburgh,', 'pittsburgh, pa')

'1931 sidney street, pittsburgh, pa 15203'

In [80]:
# Place PA in address
for i in noPA:
    df.loc[i,'address'] = df.loc[i,'address'].replace(' pittsburgh,', ' pittsburgh, pa')

In [81]:
noComma = df[~df.loc[:,'address'].str.contains(' pa') & ~df.loc[:,'address'].str.contains('pittsburgh,')].index.values

In [82]:
# Place PA in address
for i in noComma:
    df.loc[i,'address'] = df.loc[i,'address'].replace(' pittsburgh', ' pittsburgh, pa')

In [83]:
df[~df.addr2.str.contains('#') & ~df.addr2.str.contains('pittsburgh')][:50]

,structure_type,parcel,neighborhood,owner,issued,work description,permit,contractor,address,work_type,...,year,month,source,mn,clean_ward,split_addr,addr1,addr2,addr3,addr4
3,not_listed,0088r00175000000,not_listed,browns hill venture lp,2012-01-17 00:00:00,tenant fitout of 2008 sq ft retail space in on...,11-b-03148,"gem construction, inc.","4656 browns hill road,4634, pittsburgh, pa",not_listed,...,2012,january,xls,1,15,"[4656 browns hill road, 4634, pittsburgh, pa]",4656 browns hill road,4634,pittsburgh,pa
2592,not_listed,0088m00100000000,not_listed,urban redevelopment auth of pittsburgh,2013-02-06 00:00:00,"new, 2 story four (4) unit condo building (uni...",12-b-01750,kacin a richard inc,"1614 overton lane, pittsburgh, pa",not_listed,...,2013,february,API,2,14,"[1614 overton lane, a, pittsburgh, pa]",1614 overton lane,a,pittsburgh,pa
5621,not_listed,0001c01662012500,not_listed,ryan keefe,2014-01-24t00:00:00,interior condo renovations-12e,14-b-00143,randy stebler general contracting,"320 fort duquesne boulevard,12e, pittsburgh, pa",not_listed,...,2014,january,API,1,2,"[320 fort duquesne boulevard, 12e, pittsburgh...",320 fort duquesne boulevard,12e,pittsburgh,pa
5721,not_listed,0001c01662f01900,not_listed,betancourt carlos,2014-02-28t00:00:00,condo renovations,14-b-00106,fjw investment inc cabinet world,"320 fort duquesne boulevard,19f, pittsburgh, pa",not_listed,...,2014,february,API,2,2,"[320 fort duquesne boulevard, 19f, pittsburgh...",320 fort duquesne boulevard,19f,pittsburgh,pa
6619,not_listed,0010f00210000000,not_listed,housing authority city of pittsburgh,2014-05-05t00:00:00,windows & masonary repairs (building # 15),14-b-01065,air technology inc,"2505 chauncey drive,179, pittsburgh, pa",not_listed,...,2014,may,API,5,5,"[2505 chauncey drive, 179, pittsburgh, pa]",2505 chauncey drive,179,pittsburgh,pa
6620,not_listed,0010f00210000000,not_listed,housing authority city of pittsburgh,2014-05-05t00:00:00,windows & masonary repairs (building # 16),14-b-01066,air technology inc,"2525 chauncey drive,187, pittsburgh, pa",not_listed,...,2014,may,API,5,5,"[2525 chauncey drive, 187, pittsburgh, pa]",2525 chauncey drive,187,pittsburgh,pa
6628,not_listed,0010f00210000000,not_listed,housing authority city of pittsburgh,2014-05-05t00:00:00,windows & masonary repairs (building # 14),14-b-01042,air technology inc,"2469 chauncey drive,171, pittsburgh, pa",not_listed,...,2014,may,API,5,5,"[2469 chauncey drive, 171, pittsburgh, pa]",2469 chauncey drive,171,pittsburgh,pa
6722,not_listed,0010f00210000000,not_listed,housing authority city of pittsburgh,2014-05-05t00:00:00,windows & masonary repairs,14-b-00981,air technology inc,"2249 somers drive,9, pittsburgh, pa",not_listed,...,2014,may,API,5,5,"[2249 somers drive, 9, pittsburgh, pa]",2249 somers drive,9,pittsburgh,pa
6723,not_listed,0010f00210000000,not_listed,housing authority city of pittsburgh,2014-05-05t00:00:00,windows & masonary repairs,14-b-00982,air technology inc,"2267 somers drive,17, pittsburgh, pa",not_listed,...,2014,may,API,5,5,"[2267 somers drive, 17, pittsburgh, pa]",2267 somers drive,17,pittsburgh,pa
6734,not_listed,0010f00210000000,not_listed,housing authority city of pittsburgh,2014-05-05t00:00:00,windows & masonary repairs,14-b-01005,air technology inc,"2407 chauncey drive,105, pittsburgh, pa",not_listed,...,2014,may,API,5,5,"[2407 chauncey drive, 105, pittsburgh, pa]",2407 chauncey drive,105,pittsburgh,pa


In [84]:
df.loc[9646, 'address']

'3608 colby, , pittsburgh, pa 15214'

In [85]:
# Purge double commas
df.address = df.address.str.replace(',,', ',')
df.address = df.address.str.replace(', ,', ',')

In [97]:
# Redo splits
df['split_addr'] = df.loc[:,'address'].str.split(',')
df['addr1'] = df.loc[:,'split_addr'].apply(lambda x: x[0] if len(x)>0 else '')
df['addr2'] = df.loc[:,'split_addr'].apply(lambda x: x[1] if len(x)>1 else '')
df['addr3'] = df.loc[:,'split_addr'].apply(lambda x: x[2] if len(x)>2 else '')
df['addr4'] = df.loc[:,'split_addr'].apply(lambda x: x[3] if len(x)>3 else '')

In [91]:
df.loc[:,'addr2'].unique()

array([' pittsburgh', '4634', '12e', '19f', '179', '187', '171', '9',
       '17', '105', '142', '117', '129', '165', '1', '25', '153', '12h',
       '9h', '1604', '902', '11m', '1404', '21c', '20i', '1506', '9c',
       ' unit 6', ' homestead', ' #711', ' #gg', ' #jj', ' #19a', ' #ii',
       ' #hh', ' #411', ' #1006', ' #7', ' #24c', ' #4a', ' #707',
       ' carnegie', ' #u', ' #o', ' #p', ' #q', ' #r', ' #s', ' #t',
       ' #v', ' #w', ' #x', ' #y', ' #z', ' #aa', ' #24h', ' #14e',
       ' #1404', ' #165', ' #142', ' #33', ' #81', ' #57', ' #n', ' #d',
       ' #a', ' #e', ' #i', ' #g', ' #b', ' #c', ' #h', ' #l', ' #m',
       ' #j', ' #k', ' #17o', ' #906', ' #601', ' #519', ' #503', ' #505',
       ' #616', ' #15212', ' #101', ' #102', ' #103', ' #104', ' #201',
       ' #202', ' #203', ' #204', ' #301', ' #303', ' #304', ' #401',
       ' #18', ' #13', ' #414', ' #306', ' #rear', ' #512', ' #5', ' #2',
       ' #parking', ' unit 18', ' #21', ' #res', ' #55', ' #602', ' #406',

In [92]:
# replace dict
repl = {' unit ': ' # ', ' bldg ':' building ', ' rear rear':'', ' #res':'', ' #onechatham':'',
       ' ste ': ' suite ', ' #parking': '', ' #f8 ud': '#f8', '# ':'#'}

In [93]:
def complete_street(x, dct):
    for i in dct.keys():
        if i in x:
            x = x.replace(i, dct[i])
    return x

In [94]:
# Apply the func to cleanse abbreviations
df.loc[:,'address'] = df.loc[:,'address'].apply(lambda x: complete_street(x, repl))

In [95]:
df[df.address.str.contains("''")]

,structure_type,parcel,neighborhood,owner,issued,work description,permit,contractor,address,work_type,...,year,month,source,mn,clean_ward,split_addr,addr1,addr2,addr3,addr4


In [98]:
df.loc[:,'addr2'].unique()

array([' pittsburgh', '4634', '12e', '19f', '179', '187', '171', '9',
       '17', '105', '142', '117', '129', '165', '1', '25', '153', '12h',
       '9h', '1604', '902', '11m', '1404', '21c', '20i', '1506', '9c',
       ' #6', ' homestead', ' #711', ' #gg', ' #jj', ' #19a', ' #ii',
       ' #hh', ' #411', ' #1006', ' #7', ' #24c', ' #4a', ' #707',
       ' carnegie', ' #u', ' #o', ' #p', ' #q', ' #r', ' #s', ' #t',
       ' #v', ' #w', ' #x', ' #y', ' #z', ' #aa', ' #24h', ' #14e',
       ' #1404', ' #165', ' #142', ' #33', ' #81', ' #57', ' #n', ' #d',
       ' #a', ' #e', ' #i', ' #g', ' #b', ' #c', ' #h', ' #l', ' #m',
       ' #j', ' #k', ' #17o', ' #906', ' #601', ' #519', ' #503', ' #505',
       ' #616', ' #15212', ' #101', ' #102', ' #103', ' #104', ' #201',
       ' #202', ' #203', ' #204', ' #301', ' #303', ' #304', ' #401',
       ' #18', ' #13', ' #414', ' #306', ' #rear', ' #512', ' #5', ' #2',
       '', ' #21', ' #55', ' #602', ' #406', ' #b103', ' rear ', ' #f',
      

In [99]:
df[df.addr2.str.contains(r'[0-9]{3,4}') & ~df.addr2.str.contains(r'#|suite')]

,structure_type,parcel,neighborhood,owner,issued,work description,permit,contractor,address,work_type,...,year,month,source,mn,clean_ward,split_addr,addr1,addr2,addr3,addr4
3,not_listed,0088r00175000000,not_listed,browns hill venture lp,2012-01-17 00:00:00,tenant fitout of 2008 sq ft retail space in on...,11-b-03148,"gem construction, inc.","4656 browns hill road,4634, pittsburgh, pa",not_listed,...,2012,january,xls,1,15,"[4656 browns hill road, 4634, pittsburgh, pa]",4656 browns hill road,4634,pittsburgh,pa
6619,not_listed,0010f00210000000,not_listed,housing authority city of pittsburgh,2014-05-05t00:00:00,windows & masonary repairs (building # 15),14-b-01065,air technology inc,"2505 chauncey drive,179, pittsburgh, pa",not_listed,...,2014,may,API,5,5,"[2505 chauncey drive, 179, pittsburgh, pa]",2505 chauncey drive,179,pittsburgh,pa
6620,not_listed,0010f00210000000,not_listed,housing authority city of pittsburgh,2014-05-05t00:00:00,windows & masonary repairs (building # 16),14-b-01066,air technology inc,"2525 chauncey drive,187, pittsburgh, pa",not_listed,...,2014,may,API,5,5,"[2525 chauncey drive, 187, pittsburgh, pa]",2525 chauncey drive,187,pittsburgh,pa
6628,not_listed,0010f00210000000,not_listed,housing authority city of pittsburgh,2014-05-05t00:00:00,windows & masonary repairs (building # 14),14-b-01042,air technology inc,"2469 chauncey drive,171, pittsburgh, pa",not_listed,...,2014,may,API,5,5,"[2469 chauncey drive, 171, pittsburgh, pa]",2469 chauncey drive,171,pittsburgh,pa
6734,not_listed,0010f00210000000,not_listed,housing authority city of pittsburgh,2014-05-05t00:00:00,windows & masonary repairs,14-b-01005,air technology inc,"2407 chauncey drive,105, pittsburgh, pa",not_listed,...,2014,may,API,5,5,"[2407 chauncey drive, 105, pittsburgh, pa]",2407 chauncey drive,105,pittsburgh,pa
6735,not_listed,0010f00210000000,not_listed,housing authority city of pittsburgh,2014-05-05t00:00:00,windows & masonary repairs,14-b-01008,air technology inc,"2427 chauncey drive,142, pittsburgh, pa",not_listed,...,2014,may,API,5,5,"[2427 chauncey drive, 142, pittsburgh, pa]",2427 chauncey drive,142,pittsburgh,pa
6740,not_listed,0010f00210000000,not_listed,housing authority city of pittsburgh,2014-05-05t00:00:00,windows & masonary repairs,14-b-01006,air technology inc,"2409 chauncey drive,117, pittsburgh, pa",not_listed,...,2014,may,API,5,5,"[2409 chauncey drive, 117, pittsburgh, pa]",2409 chauncey drive,117,pittsburgh,pa
6741,not_listed,0010f00210000000,not_listed,housing authority city of pittsburgh,2014-05-05t00:00:00,windows & masonary repairs,14-b-01007,air technology inc,"2423 chauncey drive,129, pittsburgh, pa",not_listed,...,2014,may,API,5,5,"[2423 chauncey drive, 129, pittsburgh, pa]",2423 chauncey drive,129,pittsburgh,pa
6839,not_listed,0010f00210000000,not_listed,housing authority city of pittsburgh,2014-05-05t00:00:00,windows & masonary repairs (building # 13),14-b-01040,air technology inc,"2457 chauncey drive,165, pittsburgh, pa",not_listed,...,2014,may,API,5,5,"[2457 chauncey drive, 165, pittsburgh, pa]",2457 chauncey drive,165,pittsburgh,pa
6874,not_listed,0010f00210000000,not_listed,housing authority city of pittsburgh,2014-05-05t00:00:00,windows & masonary repairs (building # 12),14-b-01039,air technology inc,"2435 chauncey drive,153, pittsburgh, pa",not_listed,...,2014,may,API,5,5,"[2435 chauncey drive, 153, pittsburgh, pa]",2435 chauncey drive,153,pittsburgh,pa


In [100]:
idx = df[df.addr2.str.contains(r'[0-9]{3,4}') & ~df.addr2.str.contains(r'#|suite')].index.values

In [101]:
idx

array([   3, 6619, 6620, 6628, 6734, 6735, 6740, 6741, 6839, 6874, 7282,
       7639, 8031, 8625, 9034, 9058], dtype=int64)

In [102]:
# Place '#' unit signifier before second address number
for i in idx:
    df.loc[i,'addr2'] = "#"+df.loc[i,'addr2']

In [103]:
",".join(df.iloc[3,19:22])

'4656 browns hill road,#4634, pittsburgh'

In [104]:
df.loc[3,'address']

'4656 browns hill road,4634, pittsburgh, pa'

In [105]:
for i in idx:
    df.loc[i,'address'] = ",".join(df.iloc[i,19:22])

In [106]:
df[~df.addr2.str.contains(r'#|suite|pitts|home|carn|apt|rear|pa|build')]

,structure_type,parcel,neighborhood,owner,issued,work description,permit,contractor,address,work_type,...,year,month,source,mn,clean_ward,split_addr,addr1,addr2,addr3,addr4
5621,not_listed,0001c01662012500,not_listed,ryan keefe,2014-01-24t00:00:00,interior condo renovations-12e,14-b-00143,randy stebler general contracting,"320 fort duquesne boulevard,12e, pittsburgh, pa",not_listed,...,2014,january,API,1,2,"[320 fort duquesne boulevard, 12e, pittsburgh...",320 fort duquesne boulevard,12e,pittsburgh,pa
5721,not_listed,0001c01662f01900,not_listed,betancourt carlos,2014-02-28t00:00:00,condo renovations,14-b-00106,fjw investment inc cabinet world,"320 fort duquesne boulevard,19f, pittsburgh, pa",not_listed,...,2014,february,API,2,2,"[320 fort duquesne boulevard, 19f, pittsburgh...",320 fort duquesne boulevard,19f,pittsburgh,pa
6722,not_listed,0010f00210000000,not_listed,housing authority city of pittsburgh,2014-05-05t00:00:00,windows & masonary repairs,14-b-00981,air technology inc,"2249 somers drive,9, pittsburgh, pa",not_listed,...,2014,may,API,5,5,"[2249 somers drive, 9, pittsburgh, pa]",2249 somers drive,9,pittsburgh,pa
6723,not_listed,0010f00210000000,not_listed,housing authority city of pittsburgh,2014-05-05t00:00:00,windows & masonary repairs,14-b-00982,air technology inc,"2267 somers drive,17, pittsburgh, pa",not_listed,...,2014,may,API,5,5,"[2267 somers drive, 17, pittsburgh, pa]",2267 somers drive,17,pittsburgh,pa
6856,not_listed,0010f00210000000,not_listed,housing authority city of pittsburgh,2014-05-05t00:00:00,windows & masonary repairs,14-b-00980,air technology inc,"2235 somers drive,1, pittsburgh, pa",not_listed,...,2014,may,API,5,5,"[2235 somers drive, 1, pittsburgh, pa]",2235 somers drive,1,pittsburgh,pa
6857,not_listed,0010f00210000000,not_listed,housing authority city of pittsburgh,2014-05-05t00:00:00,windows & masonary repairs,14-b-00983,air technology inc,"2281 somers drive,25, pittsburgh, pa",not_listed,...,2014,may,API,5,5,"[2281 somers drive, 25, pittsburgh, pa]",2281 somers drive,25,pittsburgh,pa
6991,not_listed,0001c01662h01200,not_listed,anne and mike osborn,2014-06-06t00:00:00,interior renovations to condo unit 12 h on 12t...,14-b-01256,delaney's decorating,"320 fort duquesne boulevard,12h, pittsburgh, pa",not_listed,...,2014,june,API,6,2,"[320 fort duquesne boulevard, 12h, pittsburgh...",320 fort duquesne boulevard,12h,pittsburgh,pa
7066,not_listed,0001c01662009800,not_listed,barone alvin j jr,2014-06-13t00:00:00,interior renovations to kitchen in condo unit ...,14-b-01776,renovision,"320 fort duquesne boulevard,9h, pittsburgh, pa",not_listed,...,2014,june,API,6,2,"[320 fort duquesne boulevard, 9h, pittsburgh,...",320 fort duquesne boulevard,9h,pittsburgh,pa
8164,not_listed,0001c01662111300,not_listed,coogan david & leigh argentieri,2014-10-21t00:00:00,interior renovations to condo unit 11m in 27 s...,14-b-03439,renovision,"320 fort duquesne boulevard,11m, pittsburgh, pa",not_listed,...,2014,october,API,10,2,"[320 fort duquesne boulevard, 11m, pittsburgh...",320 fort duquesne boulevard,11m,pittsburgh,pa
8829,not_listed,0001c01662021300,not_listed,"lilan laishley, william harmon",2015-01-08t00:00:00,interior renovations to kitchen and bath in co...,15-b-00040,renovision,"320 fort duquesne boulevard,21c, pittsburgh, pa",not_listed,...,2015,january,API,1,2,"[320 fort duquesne boulevard, 21c, pittsburgh...",320 fort duquesne boulevard,21c,pittsburgh,pa


In [107]:
# Add unit '#' to remaining mixed units numbers (e.g., 12e)
idx = df[~df.addr2.str.contains(r'#|suite|pitts|home|carn|apt|rear|pa|build')].index.values

In [108]:
idx

array([ 5621,  5721,  6722,  6723,  6856,  6857,  6991,  7066,  8164,
        8829,  8920,  9321, 21816, 23473, 23994, 24485, 41191, 42622],
      dtype=int64)

In [109]:
for i in idx:
    df.loc[i,'addr2'] = "#"+df.loc[i,'addr2']

In [110]:
for i in idx:
    df.loc[i,'address'] = ",".join(df.iloc[i,19:22])

In [111]:
df[~df.addr2.str.contains(r'#|suite|pitts|home|carn|apt|rear|pa|build')]

,structure_type,parcel,neighborhood,owner,issued,work description,permit,contractor,address,work_type,...,year,month,source,mn,clean_ward,split_addr,addr1,addr2,addr3,addr4


In [112]:
# Redo splits
df['split_addr'] = df.loc[:,'address'].str.split(',')
df['addr1'] = df.loc[:,'split_addr'].apply(lambda x: x[0] if len(x)>0 else '')
df['addr2'] = df.loc[:,'split_addr'].apply(lambda x: x[1] if len(x)>1 else '')
df['addr3'] = df.loc[:,'split_addr'].apply(lambda x: x[2] if len(x)>2 else '')
df['addr4'] = df.loc[:,'split_addr'].apply(lambda x: x[3] if len(x)>3 else '')

In [113]:
df.addr2.unique()

array([' pittsburgh', '#4634', '#142', ' #6', ' homestead', ' #711',
       ' #gg', ' #jj', ' #19a', ' #ii', ' #hh', ' #411', ' #1006', ' #7',
       ' #24c', ' #4a', ' #707', ' carnegie', ' #u', ' #o', ' #p', ' #q',
       ' #r', ' #s', ' #t', ' #v', ' #w', ' #x', ' #y', ' #z', ' #aa',
       ' #24h', ' #14e', ' #1404', ' #165', ' #142', ' #33', ' #81',
       ' #57', ' #n', ' #d', ' #a', ' #e', ' #i', ' #g', ' #b', ' #c',
       ' #h', ' #l', ' #m', ' #j', ' #k', ' #17o', ' #906', ' #601',
       ' #519', ' #503', ' #505', ' #616', ' #15212', ' #101', ' #102',
       ' #103', ' #104', ' #201', ' #202', ' #203', ' #204', ' #301',
       ' #303', ' #304', ' #401', ' #18', ' #13', ' #414', ' #306',
       ' #rear', ' #512', ' #5', ' #2', ' #21', ' #55', ' #602', ' #406',
       ' #b103', ' rear ', ' #f', ' #606', ' #b102', ' #311', ' #2b',
       ' #1', ' #44', ' apt ', ' #1415', ' #2001', ' apt 1', ' #15c',
       ' #23d', ' #403', ' pittsburgh pa', ' #806', ' #4', ' #6g',
       ' #71

In [114]:
df.addr3.unique()

array([' pa', ' pittsburgh', ' pa 15217', ' pa 15212', ' pa 15206',
       ' pa 15214', ' pa 15213', ' pa 15233', ' pa 15224', ' pa 15201',
       ' pa 15222', ' pa 15203', ' pa 15216', ' pa 15219', ' pa 15211',
       ' pa 15205', ' pa 15232', ' pa 15220', ' pa 15210', ' pa 15208',
       ' pa 15204', ' pa 15226', ' pa 15207', ' pa 15218', ' pa 15227',
       ' pa 15221', ' pa 15234', ' pa 15215', ' pa 15238', ' pa 15120',
       ' pa 15106', ' pa 15235', '', ' pa 15260', ' pa 15282',
       ' pa 152071776', ' pa 15323', ' pa 150201', ' pa pa', ' pa 152101',
       ' pa 15202', ' pa 1', ' 15211', ' 15207', ' 15206', ' 15216',
       ' 15234', ' 15213', ' 15224', ' 15212', ' 15227', ' 15210',
       ' 15221', ' 15208', ' 15203', ' 15219', ' 15217', ' 15233',
       ' 15235', ' 15232'], dtype=object)

In [115]:
df[df.address.str.contains(r' pa 1$')]

,structure_type,parcel,neighborhood,owner,issued,work description,permit,contractor,address,work_type,...,year,month,source,mn,clean_ward,split_addr,addr1,addr2,addr3,addr4
47692,not_listed,0053b00100000000,not_listed,city of pittsburgh,2018-09-26t00:00:00,new hazardous exhaust systems/hvac systems rel...,18-m-00097,scalise industries,"431 hamerschlag drive, pittsburgh, pa 1",not_listed,...,2018,september,API,9,14,"[431 hamerschlag drive, pittsburgh, pa 1]",431 hamerschlag drive,pittsburgh,pa 1,


In [116]:
df.loc[47692,'address'] = '431 hamerschlag drive, pittsburgh, pa'

In [117]:
idx = df[df.address.str.contains(r'[0-9]{6,}')].index.values

In [118]:
df.loc[idx]['addr3']

20830     pa 152071776
24984     pa 152071776
25206     pa 152071776
31093        pa 150201
41088        pa 152101
Name: addr3, dtype: object

In [119]:
for i in idx:
    df.loc[i, 'addr3'] = df.loc[i, 'addr3'].split()[0] + " " + df.loc[i, 'addr3'].split()[1][:5]

In [120]:
df[df.address.str.contains(r'[0-9]{6,}')]

,structure_type,parcel,neighborhood,owner,issued,work description,permit,contractor,address,work_type,...,year,month,source,mn,clean_ward,split_addr,addr1,addr2,addr3,addr4
20830,not_listed,0011r00004000001,not_listed,almono lp,6/2/2016,construction on new storage building (148ft by...,16-b-01162,franjo construction corp,"4936 gloster street, pittsburgh, pa 152071776",not_listed,...,2016,june,xslx,6,15,"[4936 gloster street, pittsburgh, pa 152071776]",4936 gloster street,pittsburgh,pa 15207,
24984,not_listed,0011r00004000001,not_listed,almono lp,NaN,electric associated with 16-b-01161 - service ...,16-e-01871,NaN,"4964 gloster street, pittsburgh, pa 152071776",not_listed,...,2016,november,API,11,15,"[4964 gloster street, pittsburgh, pa 152071776]",4964 gloster street,pittsburgh,pa 15207,
25206,not_listed,0011r00004000001,not_listed,almono lp,NaN,"revised: constriction of new, one story office...",16-b-01161,franjo construction corp,"4964 gloster street, pittsburgh, pa 152071776",not_listed,...,2016,november,API,11,15,"[4964 gloster street, pittsburgh, pa 152071776]",4964 gloster street,pittsburgh,pa 15207,
31093,not_listed,0049f00052000000,not_listed,jmt development inc,5/3/2017,(amendment to 16-m-00293) install (2) furnaces...,17-m-00711,day & night heating & air,"4063 street johns way, pittsburgh, pa 150201",not_listed,...,2017,may,xslx,5,9,"[4063 street johns way, pittsburgh, pa 150201]",4063 street johns way,pittsburgh,pa 15020,
41088,not_listed,0049c00266000000,not_listed,paul yanulavich,3/8/2018,"replace existing windows at a two-story, singl...",18-b-00758,thermo twin industries,"4611 bancroft street, pittsburgh, pa 152101",not_listed,...,2018,march,xslx,3,9,"[4611 bancroft street, pittsburgh, pa 152101]",4611 bancroft street,pittsburgh,pa 15210,


In [121]:
for i in idx:
    df.loc[i,'address'] = ",".join(df.iloc[i,19:22])

In [122]:
df.addr3.unique()

array([' pa', ' pittsburgh', ' pa 15217', ' pa 15212', ' pa 15206',
       ' pa 15214', ' pa 15213', ' pa 15233', ' pa 15224', ' pa 15201',
       ' pa 15222', ' pa 15203', ' pa 15216', ' pa 15219', ' pa 15211',
       ' pa 15205', ' pa 15232', ' pa 15220', ' pa 15210', ' pa 15208',
       ' pa 15204', ' pa 15226', ' pa 15207', ' pa 15218', ' pa 15227',
       ' pa 15221', ' pa 15234', ' pa 15215', ' pa 15238', ' pa 15120',
       ' pa 15106', ' pa 15235', '', ' pa 15260', ' pa 15282', 'pa 15207',
       ' pa 15323', 'pa 15020', ' pa pa', 'pa 15210', ' pa 15202',
       ' pa 1', ' 15211', ' 15207', ' 15206', ' 15216', ' 15234',
       ' 15213', ' 15224', ' 15212', ' 15227', ' 15210', ' 15221',
       ' 15208', ' 15203', ' 15219', ' 15217', ' 15233', ' 15235',
       ' 15232'], dtype=object)

In [123]:
df.address = df.address.str.replace(' pa pa', ' pa')

In [124]:
# Redo splits
df['split_addr'] = df.loc[:,'address'].str.split(',')
df['addr1'] = df.loc[:,'split_addr'].apply(lambda x: x[0] if len(x)>0 else '')
df['addr2'] = df.loc[:,'split_addr'].apply(lambda x: x[1] if len(x)>1 else '')
df['addr3'] = df.loc[:,'split_addr'].apply(lambda x: x[2] if len(x)>2 else '')
df['addr4'] = df.loc[:,'split_addr'].apply(lambda x: x[3] if len(x)>3 else '')

In [125]:
df.addr4.unique()

array(['', ' pa 15234', ' pa 15232', ' pa 15222', ' pa 15201',
       ' pa 15213', ' pa 15210', ' pa 15211', ' pa 15219', ' pa 15206',
       ' pa 15217', ' pa 15208', ' pa', ' pa 15207', ' pa 15205',
       ' pa 15212', ' pa 15203', ' pa 15233', ' pa 15221', ' pa 15224',
       ' pa 15226'], dtype=object)

In [126]:
df

,structure_type,parcel,neighborhood,owner,issued,work description,permit,contractor,address,work_type,...,year,month,source,mn,clean_ward,split_addr,addr1,addr2,addr3,addr4
0,not_listed,0048s00284000000,not_listed,pfohl donald e & lucy g (w),2012-01-30 00:00:00,interior fitout for yoga studio\n-pittsburgh b...,11-b-03000,nh construction llc,"3418 penn avenue, pittsburgh, pa",not_listed,...,2012,january,xls,1,6,"[3418 penn avenue, pittsburgh, pa]",3418 penn avenue,pittsburgh,pa,
1,not_listed,0006r00206000000,not_listed,p7 investments lp,2012-01-04 00:00:00,new 3 story single family dwelling with 2 car ...,11-b-03019,heartland homes inc,"435 sweetbriar street, pittsburgh, pa",not_listed,...,2012,january,xls,1,19,"[435 sweetbriar street, pittsburgh, pa]",435 sweetbriar street,pittsburgh,pa,
2,not_listed,0006r00206000000,not_listed,dugan timothy,2012-01-04 00:00:00,new 3 story single family dwelling with 2 car ...,11-b-03021,heartland homes inc,"437 sweetbriar street 435, pittsburgh, pa",not_listed,...,2012,january,xls,1,19,"[437 sweetbriar street 435, pittsburgh, pa]",437 sweetbriar street 435,pittsburgh,pa,
3,not_listed,0088r00175000000,not_listed,browns hill venture lp,2012-01-17 00:00:00,tenant fitout of 2008 sq ft retail space in on...,11-b-03148,"gem construction, inc.","4656 browns hill road,#4634, pittsburgh",not_listed,...,2012,january,xls,1,15,"[4656 browns hill road, #4634, pittsburgh]",4656 browns hill road,#4634,pittsburgh,
4,not_listed,0027m00104000000,not_listed,mellon institute,2012-01-19 00:00:00,interior renovations to 2200 sq ft as lab spac...,11-b-03196,mosites construction,"4400 fifth avenue, pittsburgh, pa",not_listed,...,2012,january,xls,1,4,"[4400 fifth avenue, pittsburgh, pa]",4400 fifth avenue,pittsburgh,pa,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71205,residential,0084-m-00316-0000-00,shadyside,horen james gorman,2021-01-29t00:00:00,furnace & ductwork,mp-2021-01422,fazio heating & cooling llc,"323 denniston street, pittsburgh, pa 15206",addition / alteration,...,2021,january,API,1,7,"[323 denniston street, pittsburgh, pa 15206]",323 denniston street,pittsburgh,pa 15206,
71206,commercial,0025-k-00050-0000-00,strip district,2614 smallman associates lp,2021-01-29t00:00:00,new exterior mechanical equipment and interior...,mp-2020-18973,w g tomko inc,"211 26th street, pittsburgh, pa 15222",addition / alteration,...,2021,january,API,1,2,"[211 26th street, pittsburgh, pa 15222]",211 26th street,pittsburgh,pa 15222,
71207,commercial,0084-b-00200-0000-00,east liberty,alpha 4 lp,2021-01-29t00:00:00,installation of multiple split system units an...,mp-2020-19379,lakewood mechanical,"5900 penn avenue, pittsburgh, pa 15206",addition / alteration,...,2021,january,API,1,8,"[5900 penn avenue, pittsburgh, pa 15206]",5900 penn avenue,pittsburgh,pa 15206,
71208,commercial,0083-p-00241-0000-00,east liberty,arnold babar lp,2021-01-29t00:00:00,use of 1484 sf on partial second floor as offi...,mp-2020-19381,lakewood mechanical,"211 north whitfield street, pittsburgh, pa 15206",minor alteration,...,2021,january,API,1,11,"[211 north whitfield street, pittsburgh, pa ...",211 north whitfield street,pittsburgh,pa 15206,


In [145]:
df.to_csv('working_v2.csv', index=False)